In [53]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [149]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [150]:
import evaluate

In [151]:
def evaluate_mine(hypotheses, texts, gt_table):
    gts = defaultdict(list)
    for r, row in gt_table.iterrows():
        gts[row.one_text].append(row.parents_list)
    aps = pd.Series([evaluate.compute_ap(gts[w], hyps, k=10) for w, hyps in zip(texts, hypotheses)])
    return aps, gts

In [3]:
with open('cache/wiki_test_0_1000.pkl', 'rb') as f:
    docs_test = pickle.load(f)

In [4]:
len(docs_test)

762

In [12]:
docs_test[:3]

['Абдоминопластика  (от лат. abdomen «живот») — объёмное оперативное вмешательство, осуществляемое с целью восстановления эстетических пропорций живота. Абдоминопластика направлена на иссечение избытков кожи и жировых отложений, образовавшихся после родов или в силу особенностей строения организма. Как правило, большая часть пациентов — женщины после родов.\n',
 'Сорбенты (от лат. sorbens — поглощающий) — твердые тела или жидкости, избирательно поглощающие (сорбирующие) из окружающей среды газы, пары или растворённые вещества. В зависимости от характера сорбции различают абсорбенты — тела, образующие с поглощённым веществом твёрдый или жидкий раствор, адсорбенты — тела, поглощающие (сгущающие) вещество на своей (обычно сильно развитой) поверхности, и химические поглотители, которые связывают поглощаемое вещество, вступая с ним в химическое взаимодействие. Отдельную группу составляют ионообменные сорбенты (иониты), поглощающие из растворов ионы одного типа с выделением в раствор эквивал

In [17]:
public_test_nouns = pd.read_csv('../data/public_test/nouns_public.tsv', header=None, encoding='utf-8')
public_test_nouns.columns = ['noun']

In [18]:
public_test_nouns['definition'] = docs_test

In [19]:
pd.options.display.max_colwidth = 500

In [20]:
public_test_nouns.sample(3)

,noun,definition
447,ПАРЕЗ,"Парез (от др.-греч. πάρεσις [парэсис] «ослабление») — неврологический синдром, снижение силы мышц, обусловленное поражением двигательного пути нервной системы или периферического нерва.\n"
638,ТАРАНТЕЛЛА,"Тарантелла (итал. Tarantella) — итальянский народный танец в сопровождении гитары, тамбурина (он же бубен) и кастаньет (в Сицилии), музыкальный размер — 6/8, 3/8. С историей тарантеллы связано много легенд. Начиная с XV века в течение двух столетий тарантелла считалась единственным средством излечения «тарантизма» — безумия, вызываемого, как полагали, укусом тарантула (название паука тарантул, также как и танца, производят от названия южноитальянского города Таранто). «Таково очевидно происх..."
692,ФРАМУГА,"Фрамуга — горизонтальная верхняя глухая часть окна, боковая или верхняя часть проема двери.\n"


**Бейзлайн**: находим все синсеты, слова из которых входят в данный текст после первого тире. 

In [25]:
l, r = public_test_nouns.definition[293].split('—', maxsplit=1)
print(l)
print(r)

Криптон 
  химический элемент с атомным номером 36[4]. Принадлежит к 18-й группе периодической таблицы химических элементов (по устаревшей короткой форме периодической системы принадлежит к главной подгруппе VIII группы, или к группе VIIIA), находится в четвёртом периоде таблицы. Атомная масса элемента 83,798(2) а. е. м.[1].  Обозначается символом Kr (от лат. Krypton). Простое вещество криптон — инертный одноатомный газ без цвета, вкуса и запаха.



In [38]:
from pymorphy2 import MorphAnalyzer
from functools import lru_cache

morph = MorphAnalyzer()

@lru_cache(maxsize=10000)
def word2lemma(word):
    parses = morph.parse(word)
    for p in parses:
        if p.normal_form:
            return p.normal_form
    return word

In [84]:
import razdel

In [88]:
list(razdel.sentenize(df.definition.iloc[0]))[0].text

'Белорусы (белор. беларусы) — восточнославянский народ (этнос и нация).'

In [83]:
sent_tokenize(df.definition.iloc[0])

['Белорусы (белор.',
 'беларусы) — восточнославянский народ (этнос и нация).',
 'Общая численность — около 10 миллионов человек.',
 'Проживают преимущественно на территории Белоруссии, где являются доминирующей национальностью (83,7 % в 2009 году).',
 'Значительное число белорусов населяют смежные с Белоруссией территории России, Польши, Украины, Латвии и Литвы, где являются национальным меньшинством.',
 'Белорусы широко расселились по территории бывшего СССР, а также мигрировали в ряд стран за его пределами (Канада, США, Германия).']

In [92]:
def prepare_definition(text, first_sentence=False):
    if first_sentence:
        text = list(razdel.sentenize(text))[0].text
    if '—' not in text:
        return ''
    l, r = text.split('—', maxsplit=1)
    prepared = ' '.join([word2lemma(w.text) for w in razdel.tokenize(r)]).upper()
    return prepared

In [95]:
prepare_definition(public_test_nouns.definition[293], first_sentence=False)

'ХИМИЧЕСКИЙ ЭЛЕМЕНТ С АТОМНЫЙ НОМЕР 36 [ 4 ] . ПРИНАДЛЕЖАТЬ К 18-ТЬ ГРУППА ПЕРИОДИЧЕСКИЙ ТАБЛИЦА ХИМИЧЕСКИЙ ЭЛЕМЕНТ ( ПО УСТАРЕВШИЙ КОРОТКИЙ ФОРМА ПЕРИОДИЧЕСКИЙ СИСТЕМА ПРИНАДЛЕЖАТЬ К ГЛАВНЫЙ ПОДГРУППА VIII ГРУППА , ИЛИ К ГРУППА VIIIA ) , НАХОДИТЬСЯ В ЧЕТЫРЕ ПЕРИОД ТАБЛИЦА . АТОМНЫЙ МАССА ЭЛЕМЕНТ 83,798 ( 2 ) А . Е . МЕТР . [ 1 ] . ОБОЗНАЧАТЬСЯ СИМВОЛ KR ( ОТ ЛАТА . KRYPTON ) . ПРОСТОЙ ВЕЩЕСТВО КРИПТОН — ИНЕРТНЫЙ ОДНОАТОМНЫЙ ГАЗ БЕЗ ЦВЕТ , ВКУС И ЗАПАХ .'

In [54]:
with open('../data/ruwordnet/synsets.N.xml', 'r', encoding='utf-8') as f:
    synsets_n_raw = xmltodict.parse(f.read(), process_namespaces=True)

id2synset = {v['@id']: v for v in synsets_n_raw['synsets']['synset']}
print(len(id2synset))
    
word2sense = defaultdict(set)
for synset_id, synset in id2synset.items():
    senses = synset['sense']
    if not isinstance(senses, list):
        senses = [senses]
    texts = {sense['#text'] for sense in senses}
    texts.add(synset['@ruthes_name'])
    for text in texts:
        word2sense[text].add(synset_id)
print(len(word2sense))


29296
86549


In [58]:
ttest_dev = pd.read_pickle('ttest_dev.pkl')
ttest_dev['definition'] = pd.read_pickle('cache/wiki_ttest_dev_0_1000.pkl')

In [57]:
ttest_test1 = pd.read_pickle('ttest_test1.pkl')
ttest_test1['definition'] = pd.read_pickle('cache/wiki_ttest_test1_0_1000.pkl')

In [105]:
synset_id = ttest_dev.sample(1).iloc[0].SYNSET_ID
print(synset_id)
df = ttest_dev[ttest_dev.SYNSET_ID==synset_id]
df

105452-N


,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list,synset_hash,one_text,definition
1220,105452-N,ТУАРЕГИ,"105451-N,100046-N",NaN,"[105451-N, 100046-N]",0.806334,ТУАРЕГИ,"Туареги (араб. طوارق‎, самоназвание — имощаг, имошаг[10]) — народ группы берберов в Мали, Нигере, Буркина-Фасо, Алжире и Ливии.\n"
1221,105452-N,ТУАРЕГИ,"100046-N,7739-N",NaN,"[100046-N, 7739-N]",0.806334,ТУАРЕГИ,"Туареги (араб. طوارق‎, самоназвание — имощаг, имошаг[10]) — народ группы берберов в Мали, Нигере, Буркина-Фасо, Алжире и Ливии.\n"


In [106]:
for pars in df.parents_list:
    print(' | '.join([id2synset[p]['@ruthes_name'] for p in pars]))

БЕРБЕРЫ | ЭТНИЧЕСКИЕ НАРОДЫ
ЭТНИЧЕСКИЕ НАРОДЫ | ЭТНИЧЕСКАЯ ОБЩНОСТЬ


In [107]:
defin = prepare_definition(df.definition.iloc[0], first_sentence=True)
print(defin)

ИМОЩАГ , ИМОШАГ [ 10 ] ) — НАРОД ГРУППА БЕРБЕР В МАЛИ , НИГЕР , БУРКИНА-ФАСО , АЛЖИР И ЛИВИЯ .


In [108]:
from tqdm.auto import trange, tqdm

In [159]:
def extract_hypotheses(dataframe, first_sentence=True):
    origins = []
    words = []
    senses = []
    targets = []
    features = []

    for synset_id in tqdm(set(dataframe.SYNSET_ID)):
        df = dataframe[dataframe.SYNSET_ID==synset_id]
        defin = prepare_definition(df.definition.iloc[0], first_sentence=first_sentence)
        if 'parents_list' in df.columns:
            true_values = {v for pars in df.parents_list for v in pars}
        else:
            true_values = set()
        for w, wsenses in word2sense.items():
            if w in defin:        
                for sense in wsenses:
                    origins.append(df.one_text.iloc[0])
                    words.append(w)
                    senses.append(sense)
                    targets.append(sense in true_values)
                    features.append({'position': defin.find(w), 'len': len(w), 'n_senses': len(wsenses)})
    features = pd.DataFrame.from_records(features)
    targets = pd.Series(targets)
    return origins, words, senses, features, targets

In [162]:
origins, words, senses, X_dev, y_dev = extract_hypotheses(ttest_dev, first_sentence=False)
print(X_dev.shape)
print(y_dev.mean(), y_dev.sum())


(9663, 3)
0.012418503570319777 120


In [163]:
from sklearn.linear_model import LogisticRegression

In [164]:
lr = LogisticRegression()
lr.fit(X_dev, y_dev);
print(lr.coef_)

[[-0.00867818  0.13265812 -0.42108646]]


In [165]:
origins, words, senses, X_test1, y_test1 = extract_hypotheses(ttest_test1, first_sentence=False)
print(X_test1.shape)
print(y_test1.mean(), y_test1.sum())


(10858, 3)
0.010130779149014552 110


In [166]:
scores = lr.predict_proba(X_test1)[:, 1]

In [167]:
from sklearn.metrics import roc_auc_score

In [168]:
print(roc_auc_score(y_test1, scores)*2-1)

0.7124369861623303


In [169]:
scores_df = pd.DataFrame({'origin': origins, 'sense': senses, 'score': scores})

In [170]:
hypos_ranked_df = scores_df.groupby('origin').apply(lambda x: x.sort_values('score', ascending=False).sense.head(10).tolist())
hypos_ranked_df

origin
АБСОЛЮТНОСТЬ            [114364-N, 6783-N, 145566-N, 511-N, 119354-N, 4226-N, 146591-N, 106488-N, 107442-N, 121686-N]
АВИАДИСПЕТЧЕР               [129838-N, 7003-N, 198-N, 3698-N, 2550-N, 6577-N, 106529-N, 106530-N, 126350-N, 145684-N]
АЗАРТ                [120522-N, 106968-N, 106651-N, 4131-N, 140086-N, 5641-N, 110249-N, 106660-N, 126367-N, 120999-N]
АЙМАРА           [105048-N, 101422-N, 109776-N, 105137-N, 104707-N, 107581-N, 103506-N, 100070-N, 126820-N, 148389-N]
АКСАЙ                    [7472-N, 8765-N, 110872-N, 9019-N, 114364-N, 3264-N, 107257-N, 100022-N, 138562-N, 108669-N]
                                                                 ...                                                 
ШАШКА                                                                                    [114364-N, 107257-N, 3264-N]
ЩАВЕЛЬ                    [6862-N, 134162-N, 4339-N, 127599-N, 761-N, 121049-N, 107445-N, 4339-N, 109156-N, 148327-N]
ЭЛЕКТРОЛИЗ                  [6052-N, 111680-N, 10

In [171]:
small_hypos = hypos_ranked_df.values.tolist()
texts = hypos_ranked_df.index.tolist()

Что ж, на бейзлайне мы получили качество 14.67%[31%|47%] (против тех 25%, что у меня на лидерборде). 

Если брать целый текст, то будет 14.2%[35%|40%]

Есть куда расти, конечно. Но уже очень впечатляет!

In [172]:
aps, _ = evaluate_mine(small_hypos, texts, ttest_test1)
print(aps.mean())
print((aps>0).mean())
print(aps[aps>0].mean())
aps.describe()

0.14201018117684783
0.3515151515151515
0.4039944809341361


count    165.000000
mean       0.142010
std        0.267568
min        0.000000
25%        0.000000
50%        0.000000
75%        0.166667
max        1.000000
dtype: float64

А что будет, если мы научимся ранжировать гипотезы идеально? Какая максимальная капасити самой матчилки-по-вхождению-строки?

Что оказывается:
* Скор 33% для первого предложения / 40% для абзаца  - это меньше, чем у лидера лидерборда, но дохуя больше, чем я выдаю сейчас
* При этом хоть один верный родитель извлекается в 38% (из первого предложения) / 44% (из первого абзаца) случаев.
* Если хоть один верный родитель извлёкся, то 90% родителей извлеклись!

In [173]:
scores_df = pd.DataFrame({'origin': origins, 'sense': senses, 'score': y_test1})
hypos_ranked_df = scores_df.groupby('origin').apply(lambda x: x.sort_values('score', ascending=False).sense.head(10).tolist())
small_hypos = hypos_ranked_df.values.tolist()
texts = hypos_ranked_df.index.tolist()

aps, _ = evaluate_mine(small_hypos, texts, ttest_test1)
print(aps.mean())
print((aps>0).mean())
print(aps[aps>0].mean())
aps.describe()

0.39898989898989895
0.44242424242424244
0.9018264840182649


count    165.000000
mean       0.398990
std        0.468412
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
dtype: float64